In [32]:
from __future__ import print_function
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.action_chains import ActionChains
import os
import logging
from parsel import Selector
from pprint import *
import time
from peewee import *
from playhouse.sqlite_ext import SqliteExtDatabase
import datetime
import re
import copy
import pandas as pd
from IPython.display import display, HTML
import psycopg2
import json

In [33]:
input_post_code = "1000AC"

In [34]:
xpath_more_information_hover = """//ul[contains(@class, "result")]/li"""

In [35]:
xpath_next_page = """//li[contains(@class, "volgende")][not(contains(@class, "not"))]"""
xpath_search_string_insert = """//*[(@id = "search-company-input")]"""
xpath_search_button = """/html/body/div[2]/main/div/div/div/div[2]/button"""

In [36]:
xpath_eight_digit_kvk = """//*[contains(concat( " ", @class, " " ), concat( " ", "meta", " " ))]//p[(((count(preceding-sibling::*) + 1) = 1) and parent::*)]/text()"""
xpath_twenty_digit_kvk = """//*[contains(concat( " ", @class, " " ), concat( " ", "result", " " ))]/@href"""
xpath_vesting_nr = """//*[contains(concat( " ", @class, " " ), concat( " ", "meta", " " ))]//p[(((count(preceding-sibling::*) + 1) = 2) and parent::*)]/text()"""
xpath_status_of_company = """//li/a/p[@class="status"]/text()"""
xpath_headquarters = """//*[contains(concat( " ", @class, " " ), concat( " ", "type", " " ))]/text()"""
xpath_main_company_name = """//h3/text()"""
xpath_bestaande_handelsnamen = """//div[@class="more-info"]/h4[contains(text(), "Bestaande")]/following-sibling::p/text()"""
xpath_statutaire_naam = """//div[@class="more-info"]/h4[contains(text(), "Statutaire")]/following-sibling::p/text()"""
xpath_second_page_link = """//*[contains(concat( " ", @class, " " ), concat( " ", "result", " " ))]/@href"""

In [37]:
db = SqliteDatabase("kvk_temp.db")
db.connect()

class BaseModel(Model):
    class Meta:
        database = db
    
class kvkItemModel(BaseModel):
    search_string = CharField()
    eight_digit_kvk = CharField(null=True, unique=True)
    twenty_digit_kvk = CharField(null=True)
    vesting_nr = CharField(null=True)
    status_of_company = CharField(null=True)
    headquarters = BooleanField(null=True)
    website = CharField(null=True)
    main_company_name = CharField(null=True)
    bestaande_handelsnamen = CharField(null=True)
    statutaire_naam = CharField(null=True)
    second_page_company_name = CharField(null=True)
    street = CharField(null=True)
    housenumber = CharField(null=True)
    housenumber_extension = CharField(null=True)
    postcode = CharField(null=True)
    city = CharField(null=True)
    extra_infomation = CharField(null=True)
    second_page_link = CharField(null=True)
        
try:
    db.drop_tables([kvkItemModel])
except:
    pass
db.create_tables([kvkItemModel])
print("="*30)
print("Sqllite Intialization Succeed !")
print("="*30)

Sqllite Intialization Succeed !


In [38]:
def web_driver_init():
    driver = webdriver.Chrome()
    driver.get("https://www.kvk.nl/orderstraat/bedrijf-kiezen/?orig=#!shop?&q=")
    time.sleep(5)
    print("="*30)
    print("Home page opened in remote browser !")
    print("="*30)
    return driver

driver = web_driver_init()

Home page opened in remote browser !


In [39]:
def insert_search_string_click_search():
    driver.find_element_by_xpath(xpath_search_string_insert).send_keys(input_post_code)
    driver.find_element_by_xpath(xpath_search_button).click()
    time.sleep(2)


In [40]:
def open_more_information():
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight*4);")
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight*4);")
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight*4);")
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight*4);")
#     more_information_elements = driver.find_elements_by_xpath(xpath_more_information_button)
#     more_information_hover = driver.find_elements_by_xpath(xpath_more_information_hover)
#     for index in range(0, len(more_information_elements)):
#         hover = ActionChains(driver).move_to_element(more_information_hover[index])
#         hover.perform()
#         try:
#             more_information_elements[index].click()
#         except:
#             pass

In [41]:
def extract_level1_company_info_on_page():
    each_company = Selector(text=driver.page_source).xpath(xpath_more_information_hover).extract()
    print("="*30)
    print("New page Opened - Level1")
    print("="*30)
    for company in each_company:
        company_obj = Selector(text=company)
        item = kvkItemModel()
        item.search_string = copy.deepcopy(input_post_code)
        item.eight_digit_kvk = str(company_obj.xpath(xpath_eight_digit_kvk).extract_first()).strip().split()[1]
        item.twenty_digit_kvk = str(company_obj.xpath(xpath_twenty_digit_kvk).extract_first()).strip().split('kvknummer=')[-1].split('&')[0]
        if len(item.twenty_digit_kvk) <= 11:
            item.twenty_digit_kvk = str(item.twenty_digit_kvk) + "0000"
        item.vesting_nr = str(company_obj.xpath(xpath_vesting_nr).extract_first()).strip().split(" ")[-1]
        if str(item.vesting_nr).startswith("Ves") == True:
            item.vesting_nr = "None"
        item.status_of_company = str(company_obj.xpath(xpath_status_of_company).extract_first()).strip()
        if company_obj.xpath(xpath_headquarters).extract_first() is None:
            item.headquarters = False
        else:
            item.headquarters = True
        item.main_company_name = str(company_obj.xpath(xpath_main_company_name).extract_first())
        item.bestaande_handelsnamen = str(company_obj.xpath(xpath_bestaande_handelsnamen).extract_first())
        item.statutaire_naam = str(company_obj.xpath(xpath_statutaire_naam).extract_first())
        item.second_page_link = "https://www.kvk.nl/"+str(company_obj.xpath(xpath_second_page_link).extract_first())
        try:
            item.save()
        except Exception as e:
            # This means we have inserted a duplicate record.
            pass

In [42]:
def has_next_page():
    html = driver.page_source
    if Selector(text=html).xpath(xpath_next_page).extract_first() is None:
        return False
    else:
        return True

In [43]:
def extract_all_level1_company_info():
    insert_search_string_click_search()
    open_more_information()
    extract_level1_company_info_on_page()
    
    while has_next_page() == True:
        driver.find_element_by_xpath(xpath_next_page).click()
        open_more_information()
        time.sleep(3)
        extract_level1_company_info_on_page()

print("="*30)
print("Scraping process started !")
print("="*30)
extract_all_level1_company_info()


Scraping process started !
New page Opened - Level1
New page Opened - Level1
New page Opened - Level1
New page Opened - Level1


In [44]:
xpath_level2_website = """//*[contains(concat( " ", @class, " " ), concat( " ", "url", " " ))]/text()"""
xpath_level2_second_page_company_name = """//*[contains(concat( " ", @class, " " ), concat( " ", "order", " " ))]//h3/text()"""
xpath_level2_extra_infomation = """//ul[@class="order"]/li/p[not(contains(@class, "type"))]/text()"""

In [45]:
errored_company = []
for company in kvkItemModel.select():
    driver.get(company.second_page_link)
    time.sleep(3)
    print("="*30)
    print("New page Opened - Level2")
    print("="*30)
    html = driver.page_source

    company_obj = Selector(text=html)
    company.website = str(company_obj.xpath(xpath_level2_website).extract_first())
    company.second_page_company_name = str(company_obj.xpath(xpath_level2_second_page_company_name).extract_first())
    company.extra_infomation = str(company_obj.xpath(xpath_level2_extra_infomation).extract())
    
    # extract the location info
    company_location_info_list = company_obj.xpath(xpath_level2_extra_infomation).extract()
    cleaned_company_location_info_list = []
    for company_location_info in company_location_info_list:
        # Find if this is a kvk
        kvk = re.search(r"(kvk +[0-9]+)", company_location_info, flags=re.IGNORECASE)
        if kvk is not None:
            continue

        # Find if this is a vesting nr
        vest = re.search(r"(Vestigingsnr.+)", company_location_info, flags=re.IGNORECASE)
        if vest is not None:
            continue

        cleaned_company_location_info_list.append(copy.deepcopy(company_location_info))
    cleaned_company_location_info_string = str()
    for cleaned_company_location_info in cleaned_company_location_info_list:
        cleaned_company_location_info_string += cleaned_company_location_info
        cleaned_company_location_info_string += " "
    
    try:
        company.street = re.search(r'([^0-9]+)', cleaned_company_location_info_string, flags=re.IGNORECASE).group(0).strip()
    except:
        # This means we are getting banned
        print("="*30)
        print("Get banned, sleep for 5 seconds.")
        print("="*30)
        errored_company.append(company)
        time.sleep(5)
        continue
    try:
        company.housenumber = re.search(r'([0-9]+)', cleaned_company_location_info_string).group(0).strip()
    except:
        pass
    try:
        company.housenumber_extension = re.search(r'([0-9]+ [A-Z]{1} )', cleaned_company_location_info_string).group(0).split()[-1]
    except:
        company.housenumber_extension = "None"
    try:
        company.postcode = re.search(r'( [\d]{4}[\S]+)', cleaned_company_location_info_string).group(0).strip()
    except:
        pass
    try:
        try:
            re.search(r'( [\d]{4}[\S]+)', cleaned_company_location_info_string).group(0).strip()
            company.city = re.sub(r'\d+[A-Z]{2} ', "", re.search(r'(( [\d]{4}[\S]+).+)', cleaned_company_location_info_string).group(0).strip())
        except:
            try:
                re.search(r'([0-9]+ [A-Z])', cleaned_company_location_info_string).group(0).split()[-1]
                company.city = re.search(r'( [A-Z]{1} [\S ]+)', cleaned_company_location_info_string).group(0).strip()[2:]
            except:
                try:
                    company.city = re.sub("\d+", "", re.search(r'( [\d]+ [\S ]+)', cleaned_company_location_info_string).group(0).strip()).strip()
                except:
                    company.city = "None"
    except:
        pass
    finally:      
        company.save()

New page Opened - Level2
New page Opened - Level2
New page Opened - Level2
New page Opened - Level2
New page Opened - Level2
New page Opened - Level2
New page Opened - Level2
New page Opened - Level2
New page Opened - Level2
New page Opened - Level2
New page Opened - Level2
Get banned, sleep for 5 seconds.
New page Opened - Level2
Get banned, sleep for 5 seconds.
New page Opened - Level2
New page Opened - Level2
New page Opened - Level2
New page Opened - Level2
New page Opened - Level2
New page Opened - Level2
New page Opened - Level2
New page Opened - Level2
New page Opened - Level2
New page Opened - Level2
New page Opened - Level2
Get banned, sleep for 5 seconds.
New page Opened - Level2
Get banned, sleep for 5 seconds.
New page Opened - Level2
New page Opened - Level2
New page Opened - Level2
New page Opened - Level2
New page Opened - Level2
New page Opened - Level2
New page Opened - Level2


In [46]:
for company in errored_company:
    driver.get(company.second_page_link)
    time.sleep(5)
    print("="*30)
    print("New page Opened (Now Fixing Corrupted Records) - Level2")
    print("="*30)
    html = driver.page_source

    company_obj = Selector(text=html)
    company.website = str(company_obj.xpath(xpath_level2_website).extract_first())
    company.second_page_company_name = str(company_obj.xpath(xpath_level2_second_page_company_name).extract_first())
    company.extra_infomation = str(company_obj.xpath(xpath_level2_extra_infomation).extract())
    
    # extract the location info
    company_location_info_list = company_obj.xpath(xpath_level2_extra_infomation).extract()
    cleaned_company_location_info_list = []
    for company_location_info in company_location_info_list:
        # Find if this is a kvk
        kvk = re.search(r"(kvk +[0-9]+)", company_location_info, flags=re.IGNORECASE)
        if kvk is not None:
            continue

        # Find if this is a vesting nr
        vest = re.search(r"(Vestigingsnr.+)", company_location_info, flags=re.IGNORECASE)
        if vest is not None:
            continue

        cleaned_company_location_info_list.append(copy.deepcopy(company_location_info))
    cleaned_company_location_info_string = str()
    for cleaned_company_location_info in cleaned_company_location_info_list:
        cleaned_company_location_info_string += cleaned_company_location_info
        cleaned_company_location_info_string += " "
    
    try:
        company.street = re.search(r'([^0-9]+)', cleaned_company_location_info_string, flags=re.IGNORECASE).group(0).strip()
    except:
        pass
    try:
        company.housenumber = re.search(r'([0-9]+)', cleaned_company_location_info_string).group(0).strip()
    except:
        pass
    try:
        company.housenumber_extension = re.search(r'([0-9]+ [A-Z]+)', cleaned_company_location_info_string).group(0).split()[-1]
    except:
        pass
    try:
        company.postcode = re.search(r'( [\d]{4}[\S]+)', cleaned_company_location_info_string).group(0).strip()
    except:
        pass
    try:
        company.city = re.sub(r'\d+[A-Z]{2} ', "", re.search(r'(( [\d]{4}[\S]+).+)', cleaned_company_location_info_string).group(0).strip())
    except:
        pass
    finally:      
        company.save()

New page Opened (Now Fixing Corrupted Records) - Level2
New page Opened (Now Fixing Corrupted Records) - Level2
New page Opened (Now Fixing Corrupted Records) - Level2
New page Opened (Now Fixing Corrupted Records) - Level2


In [47]:
log_cols = ["search_string", "8_digit_kvk","12_digit_kvk","vesting_nr","status_of_company","headquarters","website","main_company_name","bestaande_handelsnamen","statutaire_naam","second_page_company_name","street","housenumber","housenumber_extension","postcode","city","extra_infomation"]

In [48]:
df = pd.DataFrame(columns=log_cols, dtype=str)
for company in kvkItemModel.select():
    df_row = pd.DataFrame([[company.search_string,"%s\t"%company.eight_digit_kvk,"%s\t"%company.twenty_digit_kvk,"%s\t"%company.vesting_nr,company.status_of_company,company.headquarters,company.website,company.main_company_name,company.bestaande_handelsnamen,company.statutaire_naam,company.second_page_company_name,company.street,company.housenumber,company.housenumber_extension,company.postcode,company.city,company.extra_infomation]], columns=log_cols, dtype=str)
    df = df.append(df_row, ignore_index=True)

df.to_csv("file.csv")


In [49]:
print("Job Done, successfully scrapped post_code: `%s`" % input_post_code)
db.close()
driver.close()

Job Done, successfully scrapped post_code: `1000AC`
